# Preparando o Ambiente Virtual

In [1]:
# Instalando cartopy
!pip uninstall -q shapely --yes
!pip install -q shapely cartopy --no-binary shapely --no-binary cartopy

# Instalando Py-art
!pip install -q arm_pyart

In [2]:
# Montando o Drive
from google.colab import drive
drive.mount('/content/drive')
import time
dir =  '/content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Plota figura

In [ ]:
%%time
# Importa bibliotecas
import pyart # necessário so para a barra de cor
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy
import glob
import os
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader

# Informações de tamanho da matriz e CENTRO do radar
nx, ny = 480, 480
lon_radar, lat_radar = -45.972790, -23.600795

# Sistema de coordenadas
crs = ccrs.Orthographic(central_latitude=lat_radar, central_longitude=lon_radar)

# Loop dos arquivos
for arq in sorted(glob.glob(f'{dir}input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_*.PROD')): # V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200000500dBR.dpsri.PROD
#for arq in sorted(glob.glob(f'{dir}input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200000500dBR.dpsri.PROD')): # V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200000500dBR.dpsri.PROD

    print('PROCESSANDO === >>>', arq)

    # Leitura do arquivo binário
    f = open(arq, 'rb')
    data = np.fromfile(f, count=nx*ny, dtype=np.float32)
    f.close()

    # Aplica reshape na matriz de dados
    cappi = np.reshape(data, [nx, ny], order='C')

    # Inverte o eixo y
    cappi = np.flip(cappi, 0)

    # Elimina a achuva < 1 mm/h
    cappi[cappi < 1] = np.nan

    # Coordenadas
    x = np.arange(nx)*1000 - 240000
    y = x
    xx, yy = np.meshgrid(x,y)

    # Extrai a data e horário
    basename = os.path.basename(os.path.splitext(arq)[0])
    ano, mes, dia, hor, min = basename[31:35], basename[35:37], basename[37:39], basename[39:41], basename[41:43]

    # Título da figura
    titulo = f' Precipitação 3 km - FCTH: {ano}-{mes}-{dia} {hor}:{min} UTC'

    # Formato da figura
    fig = plt.figure(figsize=[8,6])
    ax=fig.add_subplot(1,1,1, projection=crs)

    # Plota o mapa, rios, litoral, etc
    #ax.add_feature(cartopy.feature.LAND)
    #ax.add_feature(cartopy.feature.OCEAN)
    #ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
    #ax.add_feature(cartopy.feature.RIVERS)
    #ax.add_feature(cartopy.feature.COASTLINE, linewidth=0.5, color='black')
    #ax.add_feature(cartopy.feature.BORDERS, linestyle=':', linewidth=0.3)

    #--------------------------------------------------------------------------------------#
    # Plota shapefile: SP
    shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/UFs/SP/SP_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.0)

    # Plota shapefile: RJ
    shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/UFs/RJ/RJ_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.0)

    # Plota shapefile: ITAJUBÁ
    shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/itajuba/itajuba.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='red', facecolor='none', linewidth=1.0)

    # Nome do arquivo shapefile que será utilizado para desenhar o contorno no mapa
    ##shapefile = ShapelyFeature(Reader("https://github.com/evmpython/shapefile/raw/refs/heads/main/estados_do_brasil/BR_UF_2019.shp").geometries(), ccrs.PlateCarree(), facecolor="none")

    # Adiciona o contorno do shapefile ao mapa de precipitação
    ##ax.add_feature(shapefile, linewidth=1, edgecolor="black")

    # Plota shapefile: ITAJUBÁ
    # Nome do arquivo shapefile que será utilizado para desenhar o contorno no mapa
    ##shapefile = ShapelyFeature(Reader("https://github.com/evmpython/shapefile/raw/refs/heads/main/itajuba/itajuba.shp").geometries(), ccrs.PlateCarree(), facecolor="none")

    # Adiciona o contorno do shapefile ao mapa de precipitação
    ##ax.add_feature(shapefile, linewidth=1, edgecolor="red")
    #--------------------------------------------------------------------------------------#

    # Extensão da imagem
    ax.set_extent((-240000, 240000, -240000, 240000), crs=crs)

    # Plota os círculos de distancia de 100 e 240km
    circ1=plt.Circle((0, 0), radius=100000, color='grey', fill=False, lw=0.5)
    circ2=plt.Circle((0, 0), radius=240000, color='grey', fill=False, lw=0.5)
    ax.add_patch(circ1)
    ax.add_patch(circ2)

    # Plota o grid
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    # Plota mapa
    vmin = 0
    vmax = 170

    levels = np.arange(vmin,vmax, 10)

    rain = ax.contourf(xx,
                       yy,
                       cappi,
                       levels=levels,
                       vmin=vmin, vmax=vmax,
                       cmap='LangRainbow12',
                       transform=crs)

    # Plota a barra de cor
    plt.colorbar(rain, label='[mm/h]', pad=0.01)
    plt.title(titulo, fontsize='x-large')

    # Salva figura
    #plt.show()
    plt.tight_layout()
    plt.savefig(f'{dir}output/cappi_rain_fcth_{ano}{mes}{dia}_{hor}{min}.png')
    plt.close()

PROCESSANDO === >>> /content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200000500dBR.dpsri.PROD
PROCESSANDO === >>> /content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200020500dBR.dpsri.PROD
PROCESSANDO === >>> /content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200040500dBR.dpsri.PROD
PROCESSANDO === >>> /content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200060600dBR.dpsri.PROD
PROCESSANDO === >>> /content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200080600dBR.dpsri.PROD
PROCESSANDO === >>> 